In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
pd.set_option('display.max_columns', None) # 展示所有列
plt.rcParams['axes.unicode_minus']=False # 正常显示符号

# 导入Spark依赖包
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# 显著性检验
from scipy.stats import ttest_ind

import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [284]:
all_data=pd.read_csv("ABtest23q4.csv",encoding='gbk')
all_data.head()

label          user_erp  if_fg first_visit_date  if_lz level_grade  \
0     B  abby.zhaodonahoo      0       2999-01-01      1          P7   
1     B        abdmjtabdt      0       2999-01-01      1          O2   
2     B         ablkmablz      0       2999-01-01      1          O2   
3     B          ablzslym      0       2999-01-01      1          O2   
4     B           adaaga1      0       2999-01-01      0               

  work_post_name                           dept_full_name     line  \
0          销售分析岗              京东集团-京东物流-国际物流事业部-美洲区-销售拓展组  国际物流事业部   
1           盘点员岗    京东集团-京东物流-供应链事业部-新疆省区-乌市亚一FDC仓B1库-在库组    供应链其他   
2          仓储组长岗    京东集团-京东物流-供应链事业部-新疆省区-乌市亚一FDC仓B3库-出库组    供应链其他   
3          仓储组长岗    京东集团-京东物流-供应链事业部-新疆省区-乌市亚一FDC仓B2库-出库组    供应链其他   
4          电话顾问岗  京东集团-京东零售-客户体验与服务部-开放服务部-新业务交付一部-快送热线一组      非物流   

  org_busi_group_1 org_busi_group_2 org_busi_group_3 zjxl main_sequnce_descr  \
0          国际物流事业部              美洲区            销售拓展组    P               专业序列   
1           供应链事业部             新疆省区      乌市亚一FDC仓B1库    O               操作序列   
2           供应链事业部             新疆省区      乌市亚一FDC仓B3库    O               操作序列   
3           供应链事业部             新疆省区      乌市亚一FDC仓B2库    O               操作序列   
4         客户体验与服务部            开放服务部          新业务交付一部  NaN                NaN   

  sequence_descr subsequence_descr first_line second_line  work_years  \
0         业务运营序列            业务支持序列         其他          职能       2.693   
1         技能作业序列            技能作业序列        供应链         常温C       2.658   
2         技能作业序列            技能作业序列        供应链         常温C       2.945   
3         技能作业序列            技能作业序列        供应链         常温C       3.849   
4            NaN               NaN        NaN         NaN       0.153   

   jd_years  excel_num  ud_num  ud_num_visit  ud_dt  ud_dt_visit max_excel_dt  \
0  2.690411          0       0             0      0            0          NaN   
1  2.654795          0       0             0      0            0          NaN   
2  2.942466          0       0             0      0            0          NaN   
3  4.846575          0       0             0      0            0          NaN   
4  0.139726          0       0             0      0            0          NaN   

   excel_dur_file_aft  excel_dur_file_all_aft  excel_dur_file_all_aft_std  \
0                 0.0                     0.0                         0.0   
1                 0.0                     0.0                         0.0   
2                 0.0                     0.0                         0.0   
3                 0.0                     0.0                         0.0   
4                 0.0                     0.0                         0.0   

   excel_dur_file_all_aft_zs  excel_dur_file_all_aft_zs_std  dtp_aft  \
0                        0.0                            0.0      0.0   
1                        0.0                            0.0      0.0   
2                        0.0                            0.0      0.0   
3                        0.0                            0.0      0.0   
4                        0.0                            0.0      0.0   

   dtp_aft_zs  file_aft  file_aft_all  file_aft_zs  file_aft_zs_all  \
0         0.0       0.0           0.0          0.0              0.0   
1         0.0       0.0           0.0          0.0              0.0   
2         0.0       0.0           0.0          0.0              0.0   
3         0.0       0.0           0.0          0.0              0.0   
4         0.0       0.0           0.0          0.0              0.0   

   excel_dur_file_aft_step1  excel_dur_file_aft_step2  \
0                       0.0                       0.0   
1                       0.0                       0.0   
2                       0.0                       0.0   
3                       0.0                       0.0   
4                       0.0                       0.0   

   excel_dur_file_all_aft_step1  excel_dur_file_all_aft_step2  dtp_aft_step

In [153]:
print(all_data[all_data.user_erp=='chengchong10'])

    label      user_erp  if_fg first_visit_date  if_lz level_grade  \
251     A  chengchong10      1       2023-09-14      1          O2   

    work_post_name                         dept_full_name   line  \
251           驻场员岗  京东集团-京东物流-供应链事业部-上海区-上海冷链医药中心-上海项目运营组  供应链其他   

    org_busi_group_1 org_busi_group_2 org_busi_group_3 zjxl  \
251           供应链事业部              上海区         上海冷链医药中心    O   

    main_sequnce_descr sequence_descr subsequence_descr first_line  \
251               操作序列         综合作业序列            综合作业序列        供应链   

    second_line  work_years  jd_years  excel_num  ud_num  ud_num_visit  ud_dt  \
251          职能       0.795  1.780822          0       5             1      5   

     ud_dt_visit max_excel_dt  excel_dur_file_aft  excel_dur_file_all_aft  \
251            5   2023-08-15            0.477239                 0.37914   

     excel_dur_file_all_aft_std  excel_dur_file_all_aft_zs  \
251                    0.472385                   0.158716   

     excel_

In [247]:
all_data.shape
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59767 entries, 0 to 59766
Data columns (total 70 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   label                          59767 non-null  object 
 1   user_erp                       59767 non-null  object 
 2   if_fg                          59767 non-null  int64  
 3   first_visit_date               59767 non-null  object 
 4   if_lz                          59767 non-null  int64  
 5   level_grade                    59606 non-null  object 
 6   work_post_name                 59767 non-null  object 
 7   dept_full_name                 59763 non-null  object 
 8   line                           59767 non-null  object 
 9   org_busi_group_1               59762 non-null  object 
 10  org_busi_group_2               59715 non-null  object 
 11  org_busi_group_3               58786 non-null  object 
 12  zjxl                           52789 non-null 

In [286]:
af1=all_data[(all_data.label=='A')&(all_data.if_lz==1)&(all_data.excel_dur_file_all_pre>0)&(~all_data.line.isin(['非物流','国际物流事业部','市场与公共事务部','技术与数据智能部']))]         
bf1=all_data[(all_data.label=='B')&(all_data.if_fg==0)&(all_data.if_lz==1)&(all_data.excel_dur_file_all_pre>0)&(~all_data.line.isin(['非物流','国际物流事业部','市场与公共事务部','技术与数据智能部']))]         
all_data = pd.concat([af1, bf1], ignore_index=True)

In [275]:
# 计算不包含任何 NaN 值的行数
num_rows_without_nan = (~all_data.isna().all(axis=1)).sum()

# 打印结果
print(f'不包含任何 NaN 值的行数: {num_rows_without_nan}')

不包含任何 NaN 值的行数: 17287


In [276]:
# 计算包含至少一个 NaN 值的行数
num_rows_with_nan = all_data.isna().any(axis=1).sum()

# 打印结果
print(f'包含至少一个 NaN 值的行数: {num_rows_with_nan}')

包含至少一个 NaN 值的行数: 1320


In [287]:
count_a = all_data['label'].value_counts().get('A', 0)
print(f"label为'A'的个数: {count_a}")
count_b = all_data['label'].value_counts().get('B', 0)
print(f"label为'B'的个数: {count_b}")

label为'A'的个数: 3589
label为'B'的个数: 13660


In [167]:
## 对三倍标准差做去除

def remove_outliers(df, column_name):
    mean = df[column_name].mean()
    std = df[column_name].std()
    
    # 计算每个数据点与均值的差值，是否在3个标准差之内
    df_filtered = df[(df[column_name] >= (mean - 3 * std)) & (df[column_name] <= (mean + 3 * std))]
    return df_filtered


# 定义要处理的字段列表
columns_to_filter = [
    'excel_num', 'ud_num', 'ud_num_visit', 'ud_dt',
    'excel_dur_file_aft', 'excel_dur_file_all_aft', 'dtp_aft', 'file_aft',
    'excel_dur_file_aft_step1', 'excel_dur_file_aft_step2',
    'excel_dur_file_all_aft_step1', 'excel_dur_file_all_aft_step2',
    'dtp_aft_step1', 'dtp_aft_step2', 'file_aft_step1', 'file_aft_step2',
    'file_aft_all_step1', 'file_aft_all_step2', 'excel_dur_file_pre',
    'excel_dur_file_all_pre', 'dtp_pre', 'file_pre', 'excel_dur_file_pre_step1',
    'excel_dur_file_pre_step2', 'excel_dur_file_all_pre_step1',
    'excel_dur_file_all_pre_step2', 'dtp_pre_step1', 'dtp_pre_step2',
    'file_pre_step1', 'file_pre_step2', 'file_pre_all_step1', 'file_pre_all_step2'
]

# 遍历字段列表，对每个字段应用remove_outliers函数
for column in columns_to_filter:
    result = remove_outliers(all_data, column)


In [168]:
# 计算不包含任何 NaN 值的行数
num_rows_without_nan = (~all_data.isna().all(axis=1)).sum()

# 打印结果
print(f'不包含任何 NaN 值的行数: {num_rows_without_nan}')

不包含任何 NaN 值的行数: 57801


In [911]:
#IQR方法（Interquartile Range）：
def remove_outliers_iqr(df, column_name):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
    return df_filtered

columns_to_filter = [
    'excel_num', 'ud_num', 'ud_num_visit', 'ud_dt',
    'excel_dur_file_aft', 'excel_dur_file_all_aft', 'dtp_aft', 'file_aft',
    'excel_dur_file_aft_step1', 'excel_dur_file_aft_step2',
    'excel_dur_file_all_aft_step1', 'excel_dur_file_all_aft_step2',
    'dtp_aft_step1', 'dtp_aft_step2', 'file_aft_step1', 'file_aft_step2',
    'file_aft_all_step1', 'file_aft_all_step2', 'excel_dur_file_pre',
    'excel_dur_file_all_pre', 'dtp_pre', 'file_pre', 'excel_dur_file_pre_step1',
    'excel_dur_file_pre_step2', 'excel_dur_file_all_pre_step1',
    'excel_dur_file_all_pre_step2', 'dtp_pre_step1', 'dtp_pre_step2',
    'file_pre_step1', 'file_pre_step2', 'file_pre_all_step1', 'file_pre_all_step2'
]

# 遍历字段列表，对每个字段应用remove_outliers函数
for column in columns_to_filter:
    result = remove_outliers_iqr(all_data, column)

In [912]:
# 计算不包含任何 NaN 值的行数
num_rows_without_nan = (~all_data.isna().all(axis=1)).sum()

# 打印结果
print(f'不包含任何 NaN 值的行数: {num_rows_without_nan}')

不包含任何 NaN 值的行数: 17287


In [718]:
#Z值
from scipy import stats

def remove_outliers_z_score(df, column_name):
    df_filtered = df[(np.abs(stats.zscore(df[column_name])) < 3)]
    return df_filtered

columns_to_filter = [
    'excel_num', 'ud_num', 'ud_num_visit', 'ud_dt',
    'excel_dur_file_aft', 'excel_dur_file_all_aft', 'dtp_aft', 'file_aft',
    'excel_dur_file_aft_step1', 'excel_dur_file_aft_step2',
    'excel_dur_file_all_aft_step1', 'excel_dur_file_all_aft_step2',
    'dtp_aft_step1', 'dtp_aft_step2', 'file_aft_step1', 'file_aft_step2',
    'file_aft_all_step1', 'file_aft_all_step2', 'excel_dur_file_pre',
    'excel_dur_file_all_pre', 'dtp_pre', 'file_pre', 'excel_dur_file_pre_step1',
    'excel_dur_file_pre_step2', 'excel_dur_file_all_pre_step1',
    'excel_dur_file_all_pre_step2', 'dtp_pre_step1', 'dtp_pre_step2',
    'file_pre_step1', 'file_pre_step2', 'file_pre_all_step1', 'file_pre_all_step2'
]

# 遍历字段列表，对每个字段应用remove_outliers函数
for column in columns_to_filter:
    result = remove_outliers_z_score(all_data, column)

In [857]:
#删除这六个字段全为0的数
columns_to_check = [
    "excel_dur_file_all_pre", "file_pre", "file_aft", 
    "dtp_pre", "dtp_aft", "gh_file_pre", 
    "file_pre_all", "file_aft_all"
]

df_filtered = all_data[~(all_data[columns_to_check] == 0).all(axis=1)]

# 保存处理后的数据到新文件
df_filtered.to_csv("filtered_data.csv", index=False, encoding='utf-8-sig')

In [858]:
# 计算不包含任何 NaN 值的行数
num_rows_without_nan = (~df_filtered.isna().all(axis=1)).sum()

# 打印结果
print(f'不包含任何 NaN 值的行数: {num_rows_without_nan}')

不包含任何 NaN 值的行数: 17287


In [483]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

def remove_outliers(data, column, method='iforest', contamination=0.16, n_estimators=330, n_neighbors=1000):
    # 将DataFrame中的指定列转换为二维数组格式，因为sklearn的模型需要这种格式
    X = data[[column]].values
    
    if method == 'iforest':
        # 使用隔离森林
        isol = IsolationForest(contamination=contamination, n_estimators=n_estimators)
        preds = isol.fit_predict(X)
    elif method == 'lof':
        # 使用局部异常因子
        lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
        preds = lof.fit_predict(X)
        # 注意：LOF不支持predict方法，fit_predict用于训练并返回每个样本的标签
    
    # 根据预测结果过滤异常值（异常值标记为-1），即保留正常数据
    mask = preds != -1
    return data[mask]

# 假设all_data是你的原始DataFrame
# 创建一个副本，以防修改原始数据
filtered_data = all_data.copy()

columns_to_filter = [
    'dtp_pre',
 #   'dtp_aft',
  #  'file_pre',
   # 'file_aft',
  #  'file_time_pre',
  #'gh_file_pre',
  #  'file_pre_all',
 #   'file_aft_all',
 #   'excel_dur_file_all_pre_avg',
#   'excel_dur_file_all_aft_avg',
    'excel_dur_file_all_pre'
]

# 遍历字段列表，对每个字段应用remove_outliers函数
for column in columns_to_filter:
    filtered_data = remove_outliers(filtered_data, column, method='iforest', contamination=0.16, n_estimators=330)  # 或者method='lof'来使用LOF

# 计算不包含任何 NaN 值的行数
num_rows_without_nan = (~filtered_data.isna().all(axis=1)).sum()

# 打印结果
print(f'不包含任何 NaN 值的行数: {num_rows_without_nan}')

不包含任何 NaN 值的行数: 12321


In [484]:
count_a = filtered_data['label'].value_counts().get('A', 0)
print(f"label为'A'的个数: {count_a}")


label为'A'的个数: 1987


In [485]:
### Q2显著性检验
# 显著性检验
def aatest(a,b,cols):
    print('字段\t\t实验组均值\t对照组均值\t p')
    for col in cols:
        _, p = ttest_ind(a[col], b[col])
        am=a[col].mean()
        bm=b[col].mean()
        print(str(col)+'\t'+str(round(am,4))+'\t'+str(round(bm,4))+'\t'+str(round(p,4)))
a = filtered_data[filtered_data['label'] == 'A']
b = filtered_data[filtered_data['label'] == 'B']
aatest(a,b,['excel_dur_file_all_pre','dtp_pre','dtp_aft','file_pre','file_aft','gh_file_pre','work_years','jd_years','file_pre_all','file_aft_all'])
#a[['dtp_pre', 'dtp_aft','file_pre','file_aft','excel_dur_file_all_pre_avg']].describe()
#b[['dtp_pre', 'dtp_aft','file_pre','file_aft','excel_dur_file_all_pre_avg']].describe()# 指定要检查的列




字段		实验组均值	对照组均值	 p
excel_dur_file_all_pre	0.3579	0.328	0.0
dtp_pre	0.5173	0.526	0.0064
dtp_aft	0.521	0.4786	0.0
file_pre	6.1221	4.6739	0.0
file_aft	5.9274	4.4483	0.0
gh_file_pre	2.5682	0.0	0.0
work_years	6.5111	7.0359	0.0
jd_years	4.6331	4.1432	0.0
file_pre_all	5.1416	3.9174	0.0
file_aft_all	4.8939	3.44	0.0


In [199]:
# Q1显著性检验
def aatest(a,b,cols):
    print('字段\t\t实验组均值\t对照组均值\t p')
    for col in cols:
        _, p = ttest_ind(a[col], b[col])
        am=a[col].mean()
        bm=b[col].mean()
        print(str(col)+'\t'+str(round(am,4))+'\t'+str(round(bm,4))+'\t'+str(round(p,4)))
a = filtered_data[filtered_data['label'] == 'A']
b = filtered_data[filtered_data['label'] == 'B']
aatest(a,b,['excel_dur_file_all_pre','dtp_pre','dtp_aft','file_pre','file_aft','file_time_pre','gh_file_pre','work_years','jd_years','level_grade_int','file_pre_all','file_aft_all'])
#a[['dtp_pre', 'dtp_aft','file_pre','file_aft','excel_dur_file_all_pre_avg']].describe()
#b[['dtp_pre', 'dtp_aft','file_pre','file_aft','excel_dur_file_all_pre_avg']].describe()


字段		实验组均值	对照组均值	 p
excel_dur_file_all_pre	0.3604	0.3795	0.0563
dtp_pre	0.4947	0.4729	0.0
dtp_aft	0.4601	0.4502	0.0274
file_pre	6.1485	4.4898	0.0
file_aft	6.2867	4.4183	0.0


KeyError: 'file_time_pre'

In [406]:
columns_to_keep = [
    'label',
    'user_erp',
    'excel_dur_file_all_pre',
    'file_pre',
    'file_aft',
    'dtp_pre',
    'dtp_aft',
    'gh_file_pre',
    'work_years',
    'jd_years',
    'file_pre_all',
    'file_aft_all'
]

# 筛选这些列
filtered_data = filtered_data[columns_to_keep]

# 保存筛选后的DataFrame到CSV文件，不包含索引，使用utf-8-sig编码
filtered_data.to_csv('test4.csv', index=False, encoding='utf-8-sig')

In [751]:
all_data.columns

Index(['label', 'user_erp', 'if_fg', 'first_visit_date', 'if_lz',
       'level_grade', 'work_post_name', 'dept_full_name', 'line',
       'org_busi_group_1', 'org_busi_group_2', 'org_busi_group_3', 'zjxl',
       'main_sequnce_descr', 'sequence_descr', 'subsequence_descr',
       'first_line', 'second_line', 'work_years', 'jd_years', 'excel_num',
       'ud_num', 'ud_num_visit', 'ud_dt', 'ud_dt_visit', 'max_excel_dt',
       'excel_dur_file_aft', 'excel_dur_file_all_aft',
       'excel_dur_file_all_aft_std', 'excel_dur_file_all_aft_zs',
       'excel_dur_file_all_aft_zs_std', 'dtp_aft', 'dtp_aft_zs', 'file_aft',
       'file_aft_all', 'file_aft_zs', 'file_aft_zs_all',
       'excel_dur_file_aft_step1', 'excel_dur_file_aft_step2',
       'excel_dur_file_all_aft_step1', 'excel_dur_file_all_aft_step2',
       'dtp_aft_step1', 'dtp_aft_step2', 'file_aft_step1', 'file_aft_step2',
       'file_aft_all_step1', 'file_aft_all_step2', 'excel_dur_file_pre',
       'excel_dur_file_all_pre', 'exce

In [934]:
a=af1[af1.user_erp.isin(filtered_data[filtered_data.group_code=='A'].sample_id)]
# b=bf3_t2[bf3_t2.user_erp.isin(ab[ab.group_code=='B'].sample_id)]
b=pd.merge(filtered_data[filtered_data.group_code=='B'],bf1,how='left',left_on='sample_id',right_on='user_erp')
aatest(a,b,['excel_dur_file_all_pre_step1','excel_dur_file_all_pre_step2','excel_dur_file_all_aft_step1','excel_dur_file_all_aft_step2','excel_dur_file_all_pre_avg','excel_dur_file_all_pre','dtp_pre','file_pre','excel_dur_file_all_aft_avg','dtp_aft','file_aft'])

print('人数：',a.user_erp.nunique(),b.user_erp.nunique())
print(a.excel_dur_file_all_aft.mean(),b.excel_dur_file_all_aft.mean())
print('预估提效：',1-a.excel_dur_file_all_aft.mean()/b.excel_dur_file_all_aft.mean())

AttributeError: 'DataFrame' object has no attribute 'group_code'

In [935]:
print(af1.columns)
print(filtered_data.columns)


Index(['label', 'user_erp', 'if_fg', 'first_visit_date', 'if_lz',
       'level_grade', 'work_post_name', 'dept_full_name', 'line',
       'org_busi_group_1', 'org_busi_group_2', 'org_busi_group_3', 'zjxl',
       'main_sequnce_descr', 'sequence_descr', 'subsequence_descr',
       'first_line', 'second_line', 'work_years', 'jd_years', 'excel_num',
       'ud_num', 'ud_num_visit', 'ud_dt', 'ud_dt_visit', 'max_excel_dt',
       'excel_dur_file_aft', 'excel_dur_file_all_aft',
       'excel_dur_file_all_aft_std', 'excel_dur_file_all_aft_zs',
       'excel_dur_file_all_aft_zs_std', 'dtp_aft', 'dtp_aft_zs', 'file_aft',
       'file_aft_all', 'file_aft_zs', 'file_aft_zs_all',
       'excel_dur_file_aft_step1', 'excel_dur_file_aft_step2',
       'excel_dur_file_all_aft_step1', 'excel_dur_file_all_aft_step2',
       'dtp_aft_step1', 'dtp_aft_step2', 'file_aft_step1', 'file_aft_step2',
       'file_aft_all_step1', 'file_aft_all_step2', 'excel_dur_file_pre',
       'excel_dur_file_all_pre', 'exce

In [ ]:
x=['实验前1期','实验前2期','实验后1期','实验后2期']
y1=a[['excel_dur_file_all_pre_step1','excel_dur_file_all_pre_step2','excel_dur_file_all_aft_step1','excel_dur_file_all_aft_step2']].mean().tolist()
y2=b[['excel_dur_file_all_pre_step1','excel_dur_file_all_pre_step2','excel_dur_file_all_aft_step1','excel_dur_file_all_aft_step2']].mean().tolist()

plt.plot(x,y1,label='A')
plt.plot(x,y2,label='B')
plt.legend()
plt.show()